In [1]:
# Install necessary packages
# pip install psola
# pip install sounddevice soundfile
# pip install pytsmod

# Import necessary packages
# import psola
import soundfile as sf
import sounddevice as sd
import soundfile
import numpy as np
import wave
import librosa
from numpy.fft import fft, ifft


In [2]:
# Function definition to play audio
def play_flac(file_path):
    with open(file_path, 'rb') as f:
        data, fs = sf.read(f)
    
    sd.play(data, fs)
    sd.wait()

# Example usage
file_path = "data/dev-clean/84/121123/84-121123-0000.flac"
play_flac(file_path)

In [3]:
# Define function to play audio
def play_audio(audio, sample_rate):
    # Play the audio
    sd.play(audio, sample_rate)
    sd.wait()  # Wait until playback is finished

# # Psola operation
# audio, sample_rate = psola.from_file(file_path, fmin=1000, fmax =1001)
# play_audio(audio, sample_rate)

In [4]:
# Transform flac to wav
audio, sample_rate = soundfile.read(file_path)
file_path_wav = file_path.replace('flac', 'wav')
soundfile.write(file_path_wav, audio, sample_rate, 'PCM_16')

play_audio(audio, sample_rate)

In [5]:
def play_wav(file_path):
    # Load audio file
    y, sr = librosa.load(file_path, sr=None)
    
    # Play audio
    sd.play(y, sr)
    sd.wait()

def add_white_noise(y, noise_level=0.05):
    # Generate white noise
    noise = np.random.normal(scale=noise_level, size=len(y))
    
    # Add white noise to the audio
    y_noisy = y + noise
    
    return y_noisy

audio, sample_rate = librosa.load(file_path_wav)
# Add white noise
y_noisy = add_white_noise(audio)

play_audio(y_noisy, sample_rate)

In [6]:
import IPython.display as ipd
import numpy as np
import scipy
from scipy.io import wavfile
from scipy import signal

def bandpass(x, lo, hi):
    X = scipy.fft.dct(x)
    N = len(X)
    X[0:int(lo*N*2)] = 0
    X[int(hi*N*2):] = 0
    return scipy.fft.idct(X)

# Read the .frac file
# original, rate = np.loadtxt(file_path), 16000  # Assuming the sample rate is 16000 Hz
original, rate = librosa.load(file_path, sr=16000)

# Display original audio
ipd.display(ipd.HTML('Original (0 to 22050 Hz)'))
ipd.display(ipd.Audio(original, rate=rate))

# Display narrowband audio
ipd.display(ipd.HTML('Narrowband (300 Hz to 3.3 kHz)'))
narrowband_audio = bandpass(original, 300/rate, 3300/rate)
ipd.display(ipd.Audio(narrowband_audio, rate=rate))

# Display wideband audio
ipd.display(ipd.HTML('Wideband (50 Hz to 7 kHz)'))
wideband_audio = bandpass(original, 50/rate, 7000/rate)
ipd.display(ipd.Audio(wideband_audio, rate=rate))

# Display superwideband audio
ipd.display(ipd.HTML('Superwideband (50 Hz to 16 kHz)'))
superwideband_audio = bandpass(original, 50/rate, 16000/rate)
ipd.display(ipd.Audio(superwideband_audio, rate=rate))

# Display fullband audio
ipd.display(ipd.HTML('Fullband (50 Hz to 22 kHz)'))
fullband_audio = bandpass(original, 50/rate, 22000/rate)
ipd.display(ipd.Audio(fullband_audio, rate=rate))

In [7]:
import numpy as np
import pytsmod as tsm
import soundfile as sf  # you can use other audio load packages.

x, sr = sf.read(file_path)
x = x.T
x_length = x.shape[-1]  # length of the audio sequence x.

s_fixed = 1.5  # stretch the audio signal 1.3x times.
s_ap = np.array([[0, x_length / 2, x_length], [0, x_length, x_length * 1.5]])  # double the first half of the audio only and preserve the other half.

x_s_fixed = tsm.wsola(x, s_fixed)
x_s_ap = tsm.wsola(x, s_ap)

play_audio(x_s_ap, 16000)

# PSOLA

In [10]:
# implemetation from https://github.com/sannawag/TD-PSOLA/blob/master/td_psola.py
def psola(signal, peaks, f_ratio):
    """
    Time-Domain Pitch Synchronous Overlap and Add
    :param signal: original time-domain signal
    :param peaks: time-domain signal peak indices
    :param f_ratio: pitch shift ratio
    :return: pitch-shifted signal
    """
    N = len(signal)
    # Interpolate
    print("test0")
    new_signal = np.zeros(N)
    print("test1")
    new_peaks_ref = np.linspace(0, len(peaks) - 1, int(len(peaks) * f_ratio))
    print("test2")
    new_peaks = np.zeros(len(new_peaks_ref)).astype(int)
    print("test")
    for i in range(len(new_peaks)):
        weight = new_peaks_ref[i] % 1
        left = np.floor(new_peaks_ref[i]).astype(int)
        right = np.ceil(new_peaks_ref[i]).astype(int)
        new_peaks[i] = int(peaks[left] * (1 - weight) + peaks[right] * weight)

    # PSOLA
    for j in range(len(new_peaks)):
        # find the corresponding old peak index
        i = np.argmin(np.abs(peaks - new_peaks[j]))
        # get the distances to adjacent peaks
        P1 = [new_peaks[j] if j == 0 else new_peaks[j] - new_peaks[j-1],
              N - 1 - new_peaks[j] if j == len(new_peaks) - 1 else new_peaks[j+1] - new_peaks[j]]
        # edge case truncation
        if peaks[i] - P1[0] < 0:
            P1[0] = peaks[i]
        if peaks[i] + P1[1] > N - 1:
            P1[1] = N - 1 - peaks[i]
        # linear OLA window
        window = list(np.linspace(0, 1, P1[0] + 1)[1:]) + list(np.linspace(1, 0, P1[1] + 1)[1:])
        # center window from original signal at the new peak
        new_signal[new_peaks[j] - P1[0]: new_peaks[j] + P1[1]] += window * signal[peaks[i] - P1[0]: peaks[i] + P1[1]]
    return new_signal

def compute_periods_per_sequence(signal, sequence, min_period, max_period):
    """
    Computes periodicity of a time-domain signal using autocorrelation
    :param sequence: analysis window length in samples. Computes one periodicity value per window
    :param min_period: smallest allowed periodicity
    :param max_period: largest allowed periodicity
    :return: list of measured periods in windows across the signal
    """
    offset = 0  # current sample offset
    periods = []  # period length of each analysis sequence
    N = len(signal)
    while offset < N:
        fourier = fft(signal[offset: offset + sequence])
        fourier[0] = 0  # remove DC component
        autoc = ifft(fourier * np.conj(fourier)).real
        autoc_peak = min_period + np.argmax(autoc[min_period: max_period])
        periods.append(autoc_peak)
        offset += sequence
    return periods

def find_peaks(signal, fs, max_hz=950, min_hz=75, analysis_win_ms=40, max_change=1.005, min_change=0.995):
    """
    Find sample indices of peaks in time-domain signal
    :param max_hz: maximum measured fundamental frequency
    :param min_hz: minimum measured fundamental frequency
    :param analysis_win_ms: window size used for autocorrelation analysis
    :param max_change: restrict periodicity to not increase by more than this ratio from the mean
    :param min_change: restrict periodicity to not decrease by more than this ratio from the mean
    :return: peak indices
    """
    N = len(signal)
    min_period = fs // max_hz
    max_period = fs // min_hz

    # compute pitch periodicity
    sequence = int(analysis_win_ms / 1000 * fs)  # analysis sequence length in samples
    periods = compute_periods_per_sequence(signal, sequence, min_period, max_period)

    # simple hack to avoid octave error: assume that the pitch should not vary much, restrict range
    mean_period = np.mean(periods)
    max_period = int(mean_period * 1.1)
    min_period = int(mean_period * 0.9)
    periods = compute_periods_per_sequence(signal, sequence, min_period, max_period)

    # find the peaks
    peaks = [np.argmax(signal[:int(periods[0]*1.1)])]
    while True:
        prev = peaks[-1]
        idx = prev // sequence  # current autocorrelation analysis window
        if prev + int(periods[idx] * max_change) >= N:
            break
        # find maximum near expected location
        peaks.append(prev + int(periods[idx] * min_change) +
                np.argmax(signal[prev + int(periods[idx] * min_change): prev + int(periods[idx] * max_change)]))
    return np.array(peaks)

def shift_pitch(signal, fs, f_ratio):
    """
    Calls psola pitch shifting algorithm
    :param signal: original signal in the time-domain
    :param fs: sample rate
    :param f_ratio: ratio by which the frequency will be shifted
    :return: pitch-shifted signal
    """
    peaks = find_peaks(signal, fs)
    new_signal = psola(signal, peaks, f_ratio)
    return new_signal

In [11]:
original, rate = librosa.load(file_path, sr=16000)
# Display original audio
ipd.display(ipd.HTML('Original (0 to 22050 Hz)'))
ipd.display(ipd.Audio(original, rate=rate))

f_ratio = 2 ** (-2 / 12)
new_signal = shift_pitch(original, rate, f_ratio)

# Display superwideband audio
ipd.display(ipd.HTML('TD-PSOLA (f-ratio 2 ** (-2 / 12))'))
ipd.display(ipd.Audio(new_signal, rate=rate))

f_ratio = 5 ** (-2 / 12)
new_signal = shift_pitch(original, rate, f_ratio)

# Display superwideband audio
ipd.display(ipd.HTML('TD-PSOLA (5 ** (-2 / 12))'))
ipd.display(ipd.Audio(new_signal, rate=rate))


f_ratio = 10 ** (-2 / 12)
new_signal = shift_pitch(original, rate, f_ratio)

# Display superwideband audio
ipd.display(ipd.HTML('TD-PSOLA (10 ** (-2 / 12))'))
ipd.display(ipd.Audio(new_signal, rate=rate))


f_ratio = 0.1
new_signal = shift_pitch(original, rate, f_ratio)

# Display superwideband audio
ipd.display(ipd.HTML('TD-PSOLA (0.5)'))
ipd.display(ipd.Audio(new_signal, rate=rate))


test0
test1
test2
test


test0
test1
test2
test


test0
test1
test2
test


test0
test1
test2
test
